### Company Information

This is the annotated version of the company info file, where each step is explained in detail.

If you wish to see the code with minimal annotation please refer to *company_info_file.py* in this project.

First, importing the necessary packages.
*part1.creds* is a local directory created to store any API key(s). This file is added to the project's *.gitignore* so the API key(s) are protected. I was generously granted premium access to the API for this project.

In [1]:
import pandas as pd
import requests
from creds import apikeys

I am gathering large amounts of data from AlphaVantage.
For more information about this API see *alphavantage.co/documentation*.
*avkey* stores the API key from AlphaVantage.

In [48]:
avkey = apikeys.avkey

In *company_info*, I gather the following information about every single company listed on NASDAQ and the New York Stock Exchange (NYSE).

- Company overview
- Income statements (quarterly)
- Balance sheet
- Cash Flow
- Earnings

Through the process of data gathering I will identify what information is useful to the project.

Defining functions to call API for each (symbol = stock symbol for individual company):

The Nasdaq website has a comprehensive listing of the securities listed on the NYSE and  NASDAQ exchanges.
These lists have been downloaded as .csv files:
- *'nasdaq_screener_nasdaq.csv'*
- *'nasdaq_screener_nyse.csv'*
These have all the ticker symbols required to get the data from AlphaVantage.

In [40]:
# importing nasdaq securities list:
nasdaq_full = pd.read_csv('data/nasdaq_screener_all.csv')

nasdaq_list = nasdaq_full
#we simply want the lists of symbols. Some of these are either holding companies, warrants, or other securities that in effect duplicates,
#since this list will be used for the API calls.

nasdaq_list['filter_out'] = nasdaq_list.Symbol.str.contains('^', regex = False)
nasdaq_list = nasdaq_list[nasdaq_list['filter_out'] == False]
#Filter for only common stock. That should return symbols for companies.
nasdaq_list['common_stock'] =  nasdaq_list.Name.str.contains('Common Stock', case=False)
nasdaq_list = nasdaq_list[nasdaq_list['common_stock'] == True]

nasdaq_list

nasdaq_list = nasdaq_list[['Symbol', 'Name', 'Sector', 'Industry']]
companies_list = nasdaq_list['Symbol']

/var/folders/f6/_wzll4lx6db_jqfj8n56h7240000gn/T/ipykernel_22715/4104675052.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nasdaq_list['common_stock'] =  nasdaq_list.Name.str.contains('Common Stock', case=False)


In [42]:
companies_list = list(companies_list)
companies_list

['A',
 'AACI',
 'AADI',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPL',
 'AAT',
 'AATC',
 'AAWW',
 'ABBV',
 'ABC',
 'ABCB',
 'ABEO',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABNB',
 'ABOS',
 'ABR',
 'ABSI',
 'ABT',
 'ABVC',
 'AC',
 'ACAB',
 'ACAC',
 'ACAD',
 'ACAH',
 'ACAX',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACER',
 'ACET',
 'ACHC',
 'ACHR',
 'ACI',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACNB',
 'ACNT',
 'ACON',
 'ACOR',
 'ACR',
 'ACRE',
 'ACRS',
 'ACRV',
 'ACRX',
 'ACT',
 'ACTG',
 'ACU',
 'ACVA',
 'ACXP',
 'ADBE',
 'ADC',
 'ADEA',
 'ADER',
 'ADES',
 'ADI',
 'ADIL',
 'ADM',
 'ADMA',
 'ADMP',
 'ADN',
 'ADP',
 'ADPT',
 'ADRA',
 'ADSK',
 'ADT',
 'ADTH',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'AE',
 'AEAE',
 'AEE',
 'AEF',
 'AEHA',
 'AEHR',
 'AEI',
 'AEIS',
 'AEL',
 'AEMD',
 'AEO',
 'AEP',
 'AERC',
 'AES',
 'AEY',
 'AEYE',
 'AFAC',
 'AFAQ',
 'AFBI',
 'AFCG',
 'AFG',
 'AFIB',
 'AFL',
 'AFRM',
 'AGAE',
 'AGCO',
 'AGEN',
 'AGFS',
 'AGFY',
 'AGIL',
 'AGIO',
 'AGLE',
 'AGM',
 'AGNC',
 'AGR',
 'A

In [45]:
#COMPANY OVERVIEW
def get_overview(symbol,apikey):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={apikey}"
    r = requests.get(url)
    return r.json()
#INCOME STATEMENTS
def get_income_statement(symbol,apikey):
    url = f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={apikey}"
    r = requests.get(url)
    return r.json()
#BALANCE SHEETS
def get_balance_sheet(symbol,apikey):
    url = f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={apikey}"
    r = requests.get(url)
    return r.json()
#CASH FLOW
def get_cash_flow(symbol,apikey):
    url = f"https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={apikey}"
    r = requests.get(url)
    return r.json()
#EARNINGS
def get_earnings(symbol,apikey):
    url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={apikey}"
    r = requests.get(url)
    return r.json()



All the data I'm gathering here comes in JSON format. I will write code to reformat the data to get the format I want.

Before gathering data on all the companies, I'll test my code on a single company, *Eversource Energy (ES)* to figure out how it works and to work with the JSON files, which are often nested. The code from this process will be used to get data from the other companies.

In [72]:
test = companies_list[0]
test

'A'

Company Overview:

In [73]:
overview_json_test = get_overview(test, avkey)

In [74]:
#JSON output looks like this:
print(overview_json_test)

{'Symbol': 'A', 'AssetType': 'Common Stock', 'Name': 'Agilent Technologies Inc', 'Description': 'Agilent Technologies, Inc. is an American analytical instrumentation development and manufacturing company that offers its products and services to markets worldwide. Its global headquarters is located in Santa Clara, California.', 'CIK': '1090872', 'Exchange': 'NYSE', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'LIFE SCIENCES', 'Industry': 'INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY & ELEC SIGNALS', 'Address': '5301 STEVENS CREEK BLVD, SANTA CLARA, CA, US', 'FiscalYearEnd': 'October', 'LatestQuarter': '2022-10-31', 'MarketCapitalization': '45922650000', 'EBITDA': '1935000000', 'PERatio': '34.8', 'PEGRatio': '2.301', 'BookValue': '17.93', 'DividendPerShare': '0.84', 'DividendYield': '0.006', 'EPS': '4.179', 'RevenuePerShareTTM': '22.9', 'ProfitMargin': '0.183', 'OperatingMarginTTM': '0.236', 'ReturnOnAssetsTTM': '0.0953', 'ReturnOnEquityTTM': '23.49', 'RevenueTTM': '6848000000', 'Gros

In [93]:
def clean_overview(json_file):
    overview = pd.json_normalize(json_file, max_level=0)
    return overview

overview1 = clean_overview(overview_json_test)
overview_columns = overview1.columns.tolist()
overview_columns

['Symbol',
 'AssetType',
 'Name',
 'Description',
 'CIK',
 'Exchange',
 'Currency',
 'Country',
 'Sector',
 'Industry',
 'Address',
 'FiscalYearEnd',
 'LatestQuarter',
 'MarketCapitalization',
 'EBITDA',
 'PERatio',
 'PEGRatio',
 'BookValue',
 'DividendPerShare',
 'DividendYield',
 'EPS',
 'RevenuePerShareTTM',
 'ProfitMargin',
 'OperatingMarginTTM',
 'ReturnOnAssetsTTM',
 'ReturnOnEquityTTM',
 'RevenueTTM',
 'GrossProfitTTM',
 'DilutedEPSTTM',
 'QuarterlyEarningsGrowthYOY',
 'QuarterlyRevenueGrowthYOY',
 'AnalystTargetPrice',
 'TrailingPE',
 'ForwardPE',
 'PriceToSalesRatioTTM',
 'PriceToBookRatio',
 'EVToRevenue',
 'EVToEBITDA',
 'Beta',
 '52WeekHigh',
 '52WeekLow',
 '50DayMovingAverage',
 '200DayMovingAverage',
 'SharesOutstanding',
 'DividendDate',
 'ExDividendDate']

In [76]:
income_json_test = get_income_statement(test,avkey)

In [77]:
#Nested JSON of income statements
print(income_json_test)

{'symbol': 'A', 'annualReports': [{'fiscalDateEnding': '2021-10-31', 'reportedCurrency': 'USD', 'grossProfit': '1460000000', 'totalRevenue': '6236000000', 'costOfRevenue': '4776000000', 'costofGoodsAndServicesSold': '2912000000', 'operatingIncome': '1347000000', 'sellingGeneralAndAdministrative': '1619000000', 'researchAndDevelopment': '441000000', 'operatingExpenses': '2305000000', 'investmentIncomeNet': '2000000', 'netInterestIncome': '-81000000', 'interestIncome': '79000000', 'interestExpense': '81000000', 'nonInterestIncome': '6304000000', 'otherNonOperatingIncome': '92000000', 'depreciation': '122000000', 'depreciationAndAmortization': '195000000', 'incomeBeforeTax': '1360000000', 'incomeTaxExpense': '150000000', 'interestAndDebtExpense': '98000000', 'netIncomeFromContinuingOperations': '1210000000', 'comprehensiveIncomeNetOfTax': '1450000000', 'ebit': '1347000000', 'ebitda': '1636000000', 'netIncome': '1210000000'}, {'fiscalDateEnding': '2020-10-31', 'reportedCurrency': 'USD', 'g

In [78]:
def clean_income(json_file):
    income = pd.json_normalize(json_file, max_level=1)
    income_q = income.drop(columns='annualReports', axis = 1)
    income_q = income_q['quarterlyReports'].to_json()
    income_q = pd.read_json(income_q, orient = 'records')#.transpose()
    income_q = income_q[0].apply(pd.Series)
    income_q['ticker'] = income.iloc[0,0]
    income_q_cols = income_q.columns.tolist()
    income_q_cols = income_q_cols[-1:] + income_q_cols[:-1]
    income_q = income_q[income_q_cols]
    return income_q

clean_income(income_json_test)


,ticker,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
0,A,2022-07-31,USD,468000000,1711000000,1243000000,779000000,411000000,412000000,116000000,...,None,48000000,397000000,68000000,19000000,329000000,300000000,416000000,464000000,329000000
1,A,2022-04-30,USD,388000000,1586000000,1198000000,746000000,360000000,386000000,115000000,...,None,51000000,333000000,59000000,21000000,274000000,239000000,354000000,405000000,274000000
2,A,2022-01-31,USD,414000000,1653000000,1239000000,764000000,376000000,417000000,117000000,...,None,51000000,319000000,36000000,21000000,283000000,267000000,340000000,391000000,283000000
3,A,2021-10-31,USD,442000000,1654000000,1212000000,760000000,395000000,389000000,116000000,...,None,51000000,448000000,6000000,21000000,442000000,614000000,395000000,520000000,442000000
4,A,2021-07-31,USD,364000000,1565000000,1201000000,734000000,336000000,403000000,113000000,...,None,54000000,327000000,63000000,21000000,264000000,262000000,348000000,402000000,264000000
5,A,2021-04-30,USD,314000000,1505000000,1191000000,708000000,288000000,420000000,109000000,...,None,45000000,273000000,57000000,20000000,216000000,233000000,293000000,338000000,216000000
6,A,2021-01-31,USD,353000000,1524000000,1171000000,710000000,328000000,407000000,103000000,...,None,45000000,312000000,24000000,24000000,288000000,341000000,331000000,376000000,288000000
7,A,2020-10-31,USD,319000000,1463000000,1144000000,695000000,299000000,387000000,102000000,...,None,46000000,283000000,61000000,19000000,222000000,176000000,299000000,348000000,222000000
8,A,2020-07-31,USD,253000000,1246000000,993000000,592000000,230000000,347000000,92000000,...,None,45000000,219000000,20000000,19000000,199000000,261000000,238000000,283000000,199000000
9,A,2020-04-30,USD,125000000,1221000000,1096000000,581000000,102000000,358000000,197000000,...,None,47000000,121000000,20000000,20000000,101000000,84000000,141000000,188000000,101000000


Company Balance Sheet:

In [79]:
balsheet_json_test = get_balance_sheet(test, avkey)

In [80]:
def clean_balsheet(json_file):
    balsheet = pd.json_normalize(json_file, max_level=1)
    balsheet_q = balsheet['quarterlyReports'].apply(pd.Series).transpose()
    balsheet_q = balsheet_q[0].apply(pd.Series)
    balsheet_q['ticker'] = balsheet.iloc[0,0]
    balsheet_q_cols = balsheet_q.columns.tolist()
    balsheet_q_cols = balsheet_q_cols[-1:] + balsheet_q_cols[:-1]
    balsheet_q = balsheet_q[balsheet_q_cols]
    return balsheet_q

clean_balsheet(balsheet_json_test)


,ticker,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,A,2022-07-31,USD,10484000000,3690000000,1071000000,1077000000,1010000000,1345000000,6952000000,...,180000000,None,2912000000,666000000,583000000,5091000000,None,139000000,3000000,296000000
1,A,2022-04-30,USD,10455000000,3643000000,1186000000,1207000000,937000000,1237000000,6812000000,...,175000000,2730000000,2905000000,624000000,603000000,5122000000,None,160000000,3000000,299000000
2,A,2022-01-31,USD,10327000000,3474000000,1113000000,1158000000,879000000,1205000000,6853000000,...,None,2730000000,2730000000,616000000,653000000,5154000000,None,159000000,3000000,300000000
3,A,2021-10-31,USD,10705000000,3799000000,1484000000,1575000000,830000000,1238000000,7090000000,...,None,2729000000,2729000000,328000000,659000000,5389000000,None,348000000,3000000,302000000
4,A,2021-07-31,USD,10491000000,3632000000,1428000000,1428000000,818000000,1122000000,6859000000,...,130000000,2728000000,2858000000,735000000,728000000,4946000000,None,90000000,3000000,303000000
5,A,2021-04-30,USD,10398000000,3514000000,1380000000,1380000000,791000000,1075000000,6884000000,...,205000000,2727000000,2932000000,701000000,726000000,4810000000,None,-12000000,3000000,303000000
6,A,2021-01-31,USD,9674000000,3483000000,1329000000,1329000000,755000000,1087000000,6191000000,...,314000000,2185000000,2499000000,556000000,609000000,4804000000,None,4000000,3000000,305000000
7,A,2020-10-31,USD,9627000000,3415000000,1441000000,1466000000,720000000,1127000000,6393000000,...,75000000,2284000000,2359000000,285000000,614000000,4873000000,None,81000000,3000000,306000000
8,A,2020-07-31,USD,9546000000,3245000000,1358000000,1358000000,746000000,930000000,6301000000,...,40000000,2283000000,2323000000,566000000,613000000,4981000000,None,130000000,3000000,309000000
9,A,2020-04-30,USD,9455000000,3171000000,1324000000,1324000000,750000000,886000000,6284000000,...,51000000,1788000000,2488000000,513000000,614000000,4768000000,None,15000000,3000000,308000000


Cash Flow

In [81]:
cashflow_json_test = get_cash_flow(test,avkey)

In [82]:
def clean_cashflow(json_file):
    cashflow = pd.json_normalize(json_file, max_level=1)
    cashflow_q = cashflow['quarterlyReports'].apply(pd.Series).transpose()
    cashflow_q = cashflow_q[0].apply(pd.Series)
    cashflow_q['ticker'] = cashflow.iloc[0,0]
    cashflow_q_cols = cashflow_q.columns.tolist()
    cashflow_q_cols = cashflow_q_cols[-1:] + cashflow_q_cols[:-1]
    cashflow_q = cashflow_q[cashflow_q_cols]
    return cashflow_q

clean_cashflow(cashflow_json_test)

,ticker,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome
0,A,2022-07-31,USD,326000000,16000000,None,161000000,207000000,80000000,82000000,...,62000000,None,None,None,None,-295000000,None,None,None,329000000
1,A,2022-04-30,USD,283000000,22000000,None,195000000,132000000,82000000,64000000,...,63000000,None,None,None,None,-234000000,None,260000000,None,274000000
2,A,2022-01-31,USD,255000000,18000000,None,47000000,100000000,82000000,75000000,...,63000000,None,None,None,None,-420000000,None,-374000000,-4000000,283000000
3,A,2021-10-31,USD,441000000,23000000,None,137000000,80000000,84000000,62000000,...,59000000,None,None,None,None,-133000000,None,55000000,-3000000,442000000
4,A,2021-07-31,USD,334000000,17000000,None,93000000,87000000,84000000,55000000,...,59000000,None,None,None,None,-87000000,None,258000000,-3000000,264000000
5,A,2021-04-30,USD,472000000,17000000,None,200000000,31000000,77000000,31000000,...,59000000,None,None,None,None,-194000000,None,203000000,0,216000000
6,A,2021-01-31,USD,238000000,19000000,None,152000000,66000000,76000000,41000000,...,59000000,None,None,None,None,-319000000,None,-111000000,9000000,288000000
7,A,2020-10-31,USD,377000000,379000000,None,114000000,90000000,76000000,27000000,...,55000000,None,None,None,None,-246000000,None,83000000,2000000,222000000
8,A,2020-07-31,USD,290000000,14000000,None,35000000,25000000,77000000,25000000,...,56000000,None,None,None,None,-9000000,None,188000000,8000000,199000000
9,A,2020-04-30,USD,313000000,22000000,None,144000000,-12000000,76000000,33000000,...,55000000,None,None,None,None,-126000000,None,92000000,-7000000,101000000


In [83]:
earnings_json_test = get_earnings(test,avkey)

In [84]:
def clean_earnings(json_file):
    earnings = pd.json_normalize(json_file, max_level=1)
    earnings_q = earnings['quarterlyEarnings'].apply(pd.Series).transpose()
    earnings_q = earnings_q[0].apply(pd.Series)
    earnings_q['ticker'] = earnings.iloc[0,0]
    earnings_q_cols = earnings_q.columns.tolist()
    earnings_q_cols = earnings_q_cols[-1:]+earnings_q_cols[:-1]
    earnings_q = earnings_q[earnings_q_cols]
    return earnings_q

clean_earnings(earnings_json_test)

,ticker,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,A,2022-10-31,2022-11-21,1.53,1.39,0.14,10.0719
1,A,2022-07-31,2022-08-16,1.34,1.2,0.14,11.6667
2,A,2022-04-30,2022-05-24,1.13,1.12,0.01,0.8929
3,A,2022-01-31,2022-02-22,1.21,1.18,0.03,2.5424
4,A,2021-10-31,2021-11-22,1.21,1.18,0.03,2.5424
...,...,...,...,...,...,...,...
87,A,2001-01-31,2001-02-20,0.51,0.45,0.06,13.3333
88,A,2000-10-31,2000-11-20,0.73,0.54,0.19,35.1852
89,A,2000-07-31,2000-08-17,0.39,0.2,0.19,95
90,A,2000-04-30,2000-05-16,0.35,0.32,0.03,9.375


In [86]:
companies_list[0:100]

['A',
 'AACI',
 'AADI',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPL',
 'AAT',
 'AATC',
 'AAWW',
 'ABBV',
 'ABC',
 'ABCB',
 'ABEO',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABNB',
 'ABOS',
 'ABR',
 'ABSI',
 'ABT',
 'ABVC',
 'AC',
 'ACAB',
 'ACAC',
 'ACAD',
 'ACAH',
 'ACAX',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACER',
 'ACET',
 'ACHC',
 'ACHR',
 'ACI',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACNB',
 'ACNT',
 'ACON',
 'ACOR',
 'ACR',
 'ACRE',
 'ACRS',
 'ACRV',
 'ACRX',
 'ACT',
 'ACTG',
 'ACU',
 'ACVA',
 'ACXP',
 'ADBE',
 'ADC',
 'ADEA',
 'ADER',
 'ADES',
 'ADI',
 'ADIL',
 'ADM',
 'ADMA',
 'ADMP',
 'ADN',
 'ADP',
 'ADPT',
 'ADRA',
 'ADSK',
 'ADT',
 'ADTH',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'AE',
 'AEAE',
 'AEE',
 'AEF',
 'AEHA',
 'AEHR',
 'AEI',
 'AEIS',
 'AEL',
 'AEMD',
 'AEO',
 'AEP',
 'AERC',
 'AES',
 'AEY',
 'AEYE',
 'AFAC',
 'AFAQ',
 'AFBI',
 'AFCG']

Try to get first 100 overviews :

In [105]:
all_overviews = pd.DataFrame()
all_overviews[overview_columns] = overview_columns

for i in companies_list:
    output = get_overview(i, avkey)
    clean_output = clean_overview(output)
    all_overviews = pd.concat([all_overviews, clean_output])

all_overviews


,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate,Information,Error Message
0,A,Common Stock,Agilent Technologies Inc,"Agilent Technologies, Inc. is an American anal...",1090872,NYSE,USD,USA,LIFE SCIENCES,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,...,1.061,161.59,112.13,137.07,129.8,295000000,2023-01-25,2022-12-30,NaN,NaN
0,AACI,Common Stock,Armada Acquisition Corp I,None,1844817,NASDAQ,USD,USA,REAL ESTATE & CONSTRUCTION,BLANK CHECKS,...,None,10.1,9.74,9.95,9.89,20709500,None,None,NaN,NaN
0,AADI,Common Stock,Aadi Bioscience Inc,None,1422142,NASDAQ,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,...,None,26.6,11,13.2,14.37,24395100,None,None,NaN,NaN
0,AAL,Common Stock,American Airlines Group,American Airlines Group Inc. is an American pu...,6201,NASDAQ,USD,USA,ENERGY & TRANSPORTATION,"AIR TRANSPORTATION, SCHEDULED",...,1.488,21.42,11.65,13.56,15.09,649901000,2020-02-19,2020-02-04,NaN,NaN
0,AAME,Common Stock,Atlantic American Corporation,Atlantic American Corporation provides life an...,8177,NASDAQ,USD,USA,FINANCE,LIFE INSURANCE,...,0.288,3.879,2.108,2.889,2.896,20367200,2022-04-27,2022-04-12,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ZUMZ,Common Stock,Zumiez Inc,"Zumiez Inc., is a specialty retailer of clothi...",1318008,NASDAQ,USD,USA,TRADE & SERVICES,RETAIL-APPAREL & ACCESSORY STORES,...,1.473,49.98,20.29,22.38,30.07,19470400,None,None,NaN,NaN
0,ZWRK,Common Stock,Z-Work Acquisition Corp,Z-Work Acquisition Corp. The company is headqu...,1828438,NASDAQ,USD,USA,REAL ESTATE & CONSTRUCTION,BLANK CHECKS,...,None,10.04,9.68,9.95,9.85,23000000,None,None,NaN,NaN
0,ZWS,Common Stock,Zurn Water Solutions Corp,"Zurn Water Solutions Corporation designs, acqu...",1439288,NYSE,USD,USA,TECHNOLOGY,GENERAL INDUSTRIAL MACHINERY & EQUIPMENT,...,1.062,37.5,21.33,24.11,28.9,177795000,2022-12-07,2022-11-17,NaN,NaN
0,ZYNE,Common Stock,Zynerba Pharmaceuticals Inc,"Zynerba Pharmaceuticals, Inc. is a specialty c...",1621443,NASDAQ,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,...,1.817,3.61,0.5,0.679,1.218,47062700,None,None,NaN,NaN


In [108]:
all_overviews.reset_index(inplace=True)

In [112]:
all_overviews.to_csv('data/company_overview_raw.csv')